In [2]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-02-05 14:15:12.539884: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-05 14:15:12.695692: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 14:15:13.580678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [4]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [5]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [6]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition import dataset
from swallowing_recognition import dence_net

In [7]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_movement_folder = directory_path / 'washino' / 'movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    

# 引数は(データ数，画像サイズ縦，画像サイズ横，色RGB，クラス数)
# train_data = dataset.DataSet(500, 224, 224, 3, 5)
# test_data = dataset.DataSet(70, 224, 224, 3, 5)

train_data = dataset.DataSet(800, 224, 224, 3, 2)
test_data = dataset.DataSet(28, 224, 224, 3, 2)

# train_data.folder_to_dataset(train_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# train_data.folder_to_dataset(train_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# train_data.folder_to_dataset(train_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# train_data.folder_to_dataset(train_movement_folder, np.array([0, 1, 0, 0, 0]), 3)
# train_data.folder_to_dataset(train_hitting_folder, np.array([1, 0, 0, 0, 0]), 4)

train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0)
train_data.folder_to_dataset(train_cough_folder, np.array(1), 400)
train_data.folder_to_dataset(train_voice_folder, np.array(1), 500)
train_data.folder_to_dataset(train_movement_folder, np.array(1), 600)
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 700)

# test_data.folder_to_dataset(test_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# test_data.folder_to_dataset(test_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# test_data.folder_to_dataset(test_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# test_data.folder_to_dataset(test_cough_folder, np.array([0, 0, 0, 1, 0]), 3)
# test_data.folder_to_dataset(test_swallowing_folder, np.array([0, 0, 1, 0, 0]), 4)

test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14)

In [17]:
# model = dence_net.DanceNet(5)

model = dence_net.DanceNet(2)

early_stopping = EarlyStopping(
    monitor='val_loss',  # 監視する値
    patience=10,         # 性能が改善されないエポック数
    verbose=1,           # 進行状況のメッセージ表示
    mode='min',          # 「減少」を監視
    restore_best_weights=True  # 最良のモデルの重みを復元
)

model_checkpoint = ModelCheckpoint(
    filepath='20240205_binary_model_best.keras',  # 保存するファイルのパス
    monitor='val_loss',        # 監視する値
    save_best_only=True,       # 最良のモデルのみを保存
    verbose=1,                 # 進行状況のメッセージ表示
    mode='min'                 # 「減少」を監視
)

# 引数は(トレーニングデータ，ラベル，epochs，batch_size)
model.training(train_data.data, train_data.labels, 50, 32, model_checkpoint = model_checkpoint)
model.evaluate(test_data.data, test_data.labels)
model.save('20240205_binary_ep50_bs32.keras')

Epoch 1/50
23/23 [==============================] - ETA: 0s - loss: 0.2712 - accuracy: 0.8889
Epoch 1: val_loss improved from inf to 0.00000, saving model to 20240205_binary_model_best.keras
23/23 [==============================] - 53s 213ms/step - loss: 0.2712 - accuracy: 0.8889 - val_loss: 5.0922e-14 - val_accuracy: 1.0000
Epoch 2/50
23/23 [==============================] - ETA: 0s - loss: 0.1822 - accuracy: 0.9333
Epoch 2: val_loss improved from 0.00000 to 0.00000, saving model to 20240205_binary_model_best.keras
23/23 [==============================] - 3s 142ms/step - loss: 0.1822 - accuracy: 0.9333 - val_loss: 4.9692e-26 - val_accuracy: 1.0000
Epoch 3/50
23/23 [==============================] - ETA: 0s - loss: 0.1907 - accuracy: 0.9250
Epoch 3: val_loss did not improve from 0.00000
23/23 [==============================] - 3s 108ms/step - loss: 0.1907 - accuracy: 0.9250 - val_loss: 0.7370 - val_accuracy: 0.8125
Epoch 4/50
23/23 [==============================] - ETA: 0s - loss: 0.1

23/23 [==============================] - 3s 110ms/step - loss: 6.5976e-04 - accuracy: 1.0000 - val_loss: 0.4503 - val_accuracy: 0.9000
Epoch 31/50
23/23 [==============================] - ETA: 0s - loss: 7.4386e-04 - accuracy: 1.0000
Epoch 31: val_loss did not improve from 0.00000
23/23 [==============================] - 3s 110ms/step - loss: 7.4386e-04 - accuracy: 1.0000 - val_loss: 0.4808 - val_accuracy: 0.9000
Epoch 32/50
23/23 [==============================] - ETA: 0s - loss: 4.3880e-04 - accuracy: 1.0000
Epoch 32: val_loss did not improve from 0.00000
23/23 [==============================] - 3s 110ms/step - loss: 4.3880e-04 - accuracy: 1.0000 - val_loss: 0.4768 - val_accuracy: 0.9000
Epoch 33/50
23/23 [==============================] - ETA: 0s - loss: 4.5127e-04 - accuracy: 1.0000
Epoch 33: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 107ms/step - loss: 4.5127e-04 - accuracy: 1.0000 - val_loss: 0.4622 - val_accuracy: 0.9000
Epoch 34/50
23/23 [

In [8]:
# model.evaluate_print()
test_data2 = dataset.DataSet(42, 224, 224, 3, 2)
test_data2.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data2.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data2.folder_to_dataset(test_voice_folder, np.array(1), 28)

In [9]:
model.evaluate(test_data2.data, test_data2.labels)

2/2 [==============================] - 0s 132ms/step - loss: 0.5006 - accuracy: 0.9286
Test accuracy:  0.9285714030265808
2/2 [==============================] - 0s 24ms/step
正しく分類されたサンプルのインデックス: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 37 38 39 40 41]
誤って分類されたサンプルのインデックス: [ 4 17 36]
サンプル 4: 正解 = 0.0, 予測 = 1
サンプル 17: 正解 = 1.0, 予測 = 0
サンプル 36: 正解 = 1.0, 予測 = 0
